In [1]:
from data_reader import InHospitalMortalityReader
import pandas as pd
import numpy as np
import matplotlib as plt
import shutil
%matplotlib inline
%reload_ext autoreload
%autoreload 2

#from Preprocessing import *
from CustomImputer import Impute
# Build readers

#val_reader = InHospitalMortalityReader(dataset_dir='/datasets/train/', listfile='/datasets/train/val_listfile.csv', period_length=48.0)


val_patients  = set()
with open(f"./resources/valset.csv", "r") as valset_file:
    for line in valset_file:
        x, y = line.split(',')
        if int(y) == 1:
            val_patients.add(x)
            
with open(f"./datasets/ihm/train/listfile.csv") as listfile:
    lines = listfile.readlines()
    header = lines[0]
    lines = lines[1:]
    
train_lines = [x for x in lines if x[:x.find("_")] not in val_patients]
val_lines = [x for x in lines if x[:x.find("_")] in val_patients]
assert len(train_lines) + len(val_lines) == len(lines)

with open(f"./datasets/ihm/train_listfile.csv", "w") as train_listfile:
    train_listfile.write(header)
    for line in train_lines:
        train_listfile.write(line)
        
with open(f"./datasets/ihm/val_listfile.csv", "w") as val_listfile:
    val_listfile.write(header)
    for line in val_lines:
        val_listfile.write(line)
        
        
shutil.copy(f"./datasets/ihm/test/listfile.csv",
            f"./datasets/ihm/test_listfile.csv")


# Load data 

Gather test and training data. 
uses one hot encoding for categorical variables. 

small_part selects only 1000 patients

In [2]:





train_reader = InHospitalMortalityReader(dataset_dir='./datasets/ihm/train/', listfile='./datasets/ihm/train_listfile.csv', period_length=48.0)
test_reader = InHospitalMortalityReader(dataset_dir='./datasets/ihm/test/', listfile='./datasets/ihm/test_listfile.csv', period_length=48.0)
val_reader = InHospitalMortalityReader(dataset_dir='./datasets/ihm/train/', listfile='./datasets/ihm/val_listfile.csv', period_length=48.0)

train_raw = train_reader._load_data( small_part=False)
test_raw = test_reader._load_data( small_part=False)
val_raw = val_reader._load_data(small_part=False)

99260_episode2_timeseries.csv could not be loaded


NameError: name 'null' is not defined

In [4]:
train_raw[0].head()

,Hours,Capillary refill rate,Diastolic blood pressure,Fraction inspired oxygen,Glascow coma scale eye opening,Glascow coma scale motor response,Glascow coma scale total,Glascow coma scale verbal response,Glucose,Heart Rate,Height,Mean blood pressure,Oxygen saturation,Respiratory rate,Systolic blood pressure,Temperature,Weight,pH,Mortality,Episode
0,1.746111,0.0,59.0,0.21,Spontaneously,Obeys Commands,15,Oriented,128.0,86,170.0,77.0,94.0,19,118.0,36.6,81.0,7.5,0,99260_episode2_timeseries.csv
1,2.196111,0.0,59.0,0.21,Spontaneously,Obeys Commands,15,Oriented,128.0,76,170.0,77.0,94.0,19,118.0,36.6,81.0,7.5,0,99260_episode2_timeseries.csv
2,2.212778,0.0,59.0,0.21,Spontaneously,Obeys Commands,15,Oriented,128.0,76,170.0,77.0,94.0,16,118.0,36.6,81.0,7.5,0,99260_episode2_timeseries.csv
3,2.279444,0.0,81,0.21,Spontaneously,Obeys Commands,15,Oriented,128.0,76,170.0,104,95.0,16,174,36.6,81.0,7.5,0,99260_episode2_timeseries.csv
4,2.496111,0.0,81,0.21,Spontaneously,Obeys Commands,15,Oriented,128.0,72,170.0,104,94.0,12,174,37.5,81.0,7.5,0,99260_episode2_timeseries.csv


# Concat data

In [25]:
def concat_data(data):
    complete_data =  pd.DataFrame(columns=data[0].columns)
    for patient in data:
        complete_data = pd.concat([complete_data,patient])
    return complete_data

train = concat_data(train_raw)
test = concat_data(test_raw)
val = concat_data(val_raw)
    

In [26]:
train.head()

,Hours,Capillary refill rate,Diastolic blood pressure,Fraction inspired oxygen,Glascow coma scale eye opening,Glascow coma scale motor response,Glascow coma scale total,Glascow coma scale verbal response,Glucose,Heart Rate,Height,Mean blood pressure,Oxygen saturation,Respiratory rate,Systolic blood pressure,Temperature,Weight,pH,Mortality,Episode
0,1.746111,0.0,59.0,0.21,Spontaneously,Obeys Commands,15,Oriented,128.0,86,170.0,77.0,94,19,118.0,36.6,81.0,7.5,0,99260_episode2_timeseries.csv
1,2.196111,0.0,59.0,0.21,Spontaneously,Obeys Commands,15,Oriented,128.0,76,170.0,77.0,94,19,118.0,36.6,81.0,7.5,0,99260_episode2_timeseries.csv
2,2.212778,0.0,59.0,0.21,Spontaneously,Obeys Commands,15,Oriented,128.0,76,170.0,77.0,94,16,118.0,36.6,81.0,7.5,0,99260_episode2_timeseries.csv
3,2.279444,0.0,81,0.21,Spontaneously,Obeys Commands,15,Oriented,128.0,76,170.0,104,95,16,174,36.6,81.0,7.5,0,99260_episode2_timeseries.csv
4,2.496111,0.0,81,0.21,Spontaneously,Obeys Commands,15,Oriented,128.0,72,170.0,104,94,12,174,37.5,81.0,7.5,0,99260_episode2_timeseries.csv


In [27]:
train.to_csv('./datasets/ihm/train.csv')
test.to_csv('./datasets/ihm/test.csv')
val.to_csv('./datasets/ihm/val.csv')

# Check for duplicate categorical values

In [38]:
print(test['Glascow coma scale verbal response'].unique())

print(test['Glascow coma scale eye opening'].unique())

print(test['Glascow coma scale motor response'].unique())

['Oriented' 'Confused' 'Inappropriate Words' 'Incomprehensible sounds'
 'No Response']
['Spontaneously' 'To Speech' 'To Pain' 'No response' 'None']
['Obeys Commands' 'Localizes Pain' 'Flex-withdraws' 'No response'
 'Abnormal Flexion' 'Abnormal extension']


In [35]:
def remove_duplicate_values(data):
    data['Glascow coma scale verbal response'].replace(['1 No Response', 'No Response-ETT','1.0 ET/Trach'], 'No Response', inplace=True)
    data['Glascow coma scale verbal response'].replace('2 Incomp sounds', 'Incomprehensible sounds', inplace=True)
    data['Glascow coma scale verbal response'].replace('3 Inapprop words', 'Inappropriate Words', inplace=True)
    data['Glascow coma scale verbal response'].replace('4 Confused', 'Confused', inplace=True)
    data['Glascow coma scale verbal response'].replace('5 Oriented', 'Oriented', inplace=True)
    
    data['Glascow coma scale eye opening'].replace('1 No Response','No response', inplace=True )
    data['Glascow coma scale eye opening'].replace('2 To pain','To Pain' , inplace=True)
    data['Glascow coma scale eye opening'].replace('3 To speech','To Speech' , inplace=True)
    data['Glascow coma scale eye opening'].replace('4 Spontaneously','Spontaneously', inplace=True )
    
    data['Glascow coma scale motor response'].replace('1 No Response','No response', inplace=True )
    data['Glascow coma scale motor response'].replace('2 Abnorm extensn','Abnormal extension', inplace=True )
    data['Glascow coma scale motor response'].replace('3 Abnorm flexion','Abnormal Flexion', inplace=True )
    data['Glascow coma scale motor response'].replace('4 Flex-withdraws','Flex-withdraws', inplace=True )
    data['Glascow coma scale motor response'].replace('5 Localizes Pain','Localizes Pain', inplace=True )
    data['Glascow coma scale motor response'].replace('6 Obeys Commands','Obeys Commands', inplace=True )
    return data
 
train = remove_duplicate_values(train) 
test = remove_duplicate_values(test)
val = remove_duplicate_values(val)
    

In [36]:
print(test['Glascow coma scale verbal response'].unique())

print(test['Glascow coma scale eye opening'].unique())

print(test['Glascow coma scale motor response'].unique())

['Oriented' 'Confused' 'Inappropriate Words' 'Incomprehensible sounds'
 'No Response']
['Spontaneously' 'To Speech' 'To Pain' 'No response' 'None']
['Obeys Commands' 'Localizes Pain' 'Flex-withdraws' 'No response'
 'Abnormal Flexion' 'Abnormal extension']


In [39]:
train.to_csv('./datasets/ihm/train.csv')
test.to_csv('./datasets/ihm/test.csv')
val.to_csv('./datasets/ihm/val.csv')

In [ ]:
def add_length(df):
    split_set = []
    num_survived = 0
    num_died = 0
    for episode, df_episode in df.groupby('Episode'):
        split_set.append(df_episode)
    for patient in split_set:
        patient['Length'] = len(patient)
    complete_data =  pd.DataFrame(columns=df.columns)
    complete_data = pd.concat(split_set)
    return complete_data

train_balanced = add_length(train_balanced)
test = add_length(test)
val = add_length(val)

In [ ]:
train_balanced.to_csv('./datasets/ihm/train_height.csv')
test.to_csv('./datasets/ihm/test_height.csv')
val.to_csv('./datasets/ihm/val_heigth.csv')